In [1]:
import os
import sys

In [2]:
sys.path.append(os.path.abspath('../src'))

In [3]:
from performance import performance
from read_and_write_docs import read_jsonl, write_jsonl

In [4]:
base_loc = "/Volumes/BCross/datasets/author_verification/training/Wiki/Qwen_2.5_1.5B/"

gen_07 = read_jsonl(f"{base_loc}/gen_t_0.7_tp_0.9/lambda_g_results.jsonl")
# gen_10 = read_jsonl(f"{base_loc}/gen_t_1.0_tp_0.9/lambda_g_results.jsonl")
gen_12 = read_jsonl(f"{base_loc}/gen_t_1.2_tp_0.9/lambda_g_results.jsonl")
gen_15 = read_jsonl(f"{base_loc}/gen_t_1.5_tp_0.9/lambda_g_results.jsonl")

wiki_base = read_jsonl("/Volumes/BCross/datasets/author_verification/lambda_g_results/Wiki_training_qwen2.5_1.5b_instruct_raw.jsonl")

In [5]:
wiki_base

,repetition,corpus,token_type,data_type,problem,known_author,unknown_author,target,score
0,1,Wiki,Qwen2.5-1.5B-Instruct,training,142.196.88.228 vs 142.196.88.228,142.196.88.228,142.196.88.228,True,24782.920438
1,1,Wiki,Qwen2.5-1.5B-Instruct,training,142.196.88.228 vs Aban1313,142.196.88.228,Aban1313,False,-7625.459464
2,1,Wiki,Qwen2.5-1.5B-Instruct,training,A_Man_In_Black vs A_Man_In_Black,A_Man_In_Black,A_Man_In_Black,True,8957.235849
3,1,Wiki,Qwen2.5-1.5B-Instruct,training,A_Man_In_Black vs Bankhallbretherton,A_Man_In_Black,Bankhallbretherton,False,-8000.678715
4,1,Wiki,Qwen2.5-1.5B-Instruct,training,Aban1313 vs Aban1313,Aban1313,Aban1313,True,6517.207140
...,...,...,...,...,...,...,...,...,...
745,5,Wiki,Qwen2.5-1.5B-Instruct,training,Haymaker vs HeadleyDown,Haymaker,HeadleyDown,False,720.970320
746,5,Wiki,Qwen2.5-1.5B-Instruct,training,HeadleyDown vs HeadleyDown,HeadleyDown,HeadleyDown,True,8547.524435
747,5,Wiki,Qwen2.5-1.5B-Instruct,training,HeadleyDown vs Hipocrite,HeadleyDown,Hipocrite,False,-11640.709076
748,5,Wiki,Qwen2.5-1.5B-Instruct,training,Hipocrite vs Hipocrite,Hipocrite,Hipocrite,True,3169.037893


In [6]:
def grouped_results(df, grouped_cols=['corpus', 'problem', 'data_type', 'target']):
    
    df_agg = (
        df
        .groupby(grouped_cols, as_index=False)
        ['score']
        .mean()
    )
    
    return df_agg

In [7]:
df_agg = grouped_results(wiki_base)

In [8]:
df_agg

,corpus,problem,data_type,target,score
0,Wiki,142.196.88.228 vs 142.196.88.228,training,True,24051.358271
1,Wiki,142.196.88.228 vs Aban1313,training,False,-7572.696933
2,Wiki,A_Man_In_Black vs A_Man_In_Black,training,True,8986.373063
3,Wiki,A_Man_In_Black vs Bankhallbretherton,training,False,-7891.412054
4,Wiki,Aban1313 vs Aban1313,training,True,5779.732507
...,...,...,...,...,...
145,Wiki,Haymaker vs HeadleyDown,training,False,855.625699
146,Wiki,HeadleyDown vs HeadleyDown,training,True,9073.383046
147,Wiki,HeadleyDown vs Hipocrite,training,False,-11853.920455
148,Wiki,Hipocrite vs Hipocrite,training,True,2562.027485


In [9]:
score_col = 'score'
target_col = 'target'
result_name = 'Experiment_1'
# result_description = 'Initial experiment with basic model'
# corpus = 'Wiki'
# data_type = 'test'
# model = 'Logistic_Regression'
keep_cols = ['corpus', 'data_type', 'description']
keep_cols = ['corpus', 'data_type']

In [10]:
result_metrics = performance(df_agg,
                             score_col,
                             target_col,
                             keep_cols = keep_cols,
                             additional_metadata={'result_name': result_name})

In [11]:
result_metrics

,result_name,corpus,data_type,Cllr,Cllr_min,EER,Mean_TRUE_LLR,Mean_FALSE_LLR,TRUE_trials,FALSE_trials,AUC,Balanced_Accuracy,Precision,Recall,F1,TP,FP,FN,TN
0,Experiment_1,Wiki,training,0.533408,0.533408,0.16,1.262599,-0.986831,75,75,0.920178,0.84,0.84,0.84,0.84,63,12,12,63
